In [56]:
from loadTDMS import *
from fft_my_0_m200 import *
import numpy as np
import pandas as pd
from dask import dataframe as dd
import matplotlib.pyplot as plt
import itertools
import os

## ファイル名変更 ex) AE_18.tdms -> AE_1.tdms
### プログラム書きやすいように

In [35]:
# os.rename(変更前ファイル、変更後ファイル)

for i in range(1261,1533,2):
    num = int((i-15)/2) - 22
    old_name = rf"Z:\100 実験データ（RAW）\20220630---切削実験---AE+切削抵抗\AE\1-5000\AE_{i}.tdms"
    new_name = rf"Z:\100 実験データ（RAW）\20220630---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    os.rename(old_name, new_name)
#     print(i,num)

# ①生データ から2^14個取ってくる
## 中間データ①

In [41]:
# AE_{num}.tdms -> AE_extract_{num}.csv
# 生データ から2^14個取ってくる

def extract_16384(num):
    load_path = rf"Z:\100 実験データ（RAW）\20220630---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    save_path = rf"Z:\200 実験データ加工\20220630\TrainingData\extract\AE_extract_{num}.csv"
    d = load_tdms(load_path, ch_num=3)
    d = d[0][5000000:5000000+16384]
    pd.DataFrame(d).to_csv(save_path)

In [45]:
for i in range(1,737):
    extract_16384(i)

# ②正規化する前のFFT
## 中間データ②作成

In [19]:
def FFT_withoutNorm(num):
    
    #データ読み込み
    d = pd.read_csv(rf'Z:\200 実験データ加工\20220630\TrainingData\extract\AE_extract_{num}.csv')
    d = d['0']
    
    #FFT条件
    dt = 1/10**6
    split_t_r = 0.1 # 1つの枠で全体のどの割合のデータを分析するか。
    overlap = 0.5  # オーバーラップ率
    window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
    y_label = "amplitude"
    y_unit = "V"
    N = len(d)
    fq = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
    

    #FFT実行
    fq_, amp_ = FFT_main(fq, d, dt, split_t_r, overlap, window_F)

    #fig保存ー正規化前
    output_FN = fr"Z:\200 実験データ加工\20220630\TrainingData\FFT\fig\FFT_{num}"
    plot_FFT(fq_, amp_, output_FN, y_label, y_unit,num)

    #dBに変換してcsv保存ー正規化前
    amp_db = pd.DataFrame({'db': 20*np.log10(amp_)})
    output_csv = fr"Z:\200 実験データ加工\20220630\TrainingData\FFT\csv\FFT_{num}.csv"
    amp_db.to_csv(output_csv, header=False, index=False)


In [20]:
%%time
for i in range(1,737):
    print(i)
    FFT_withoutNorm(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# FFT(正規化）
## 学習（trarin）データ作成

In [49]:
#学習データから最大最小を算出
#今回は2022年6月の200path-300path
def get_minmax(start,end):
    all_data = []
    for i in range(start,end+1):
        d = pd.read_csv(rf"Z:\200 実験データ加工\20220630\TrainingData\FFT\csv\FFT_{i}.csv")
        d = d.to_numpy()
        d = np.ravel(d)
        all_data.append(d[:])
    all_data = np.array(all_data)
    all_data = all_data.ravel()
    min_d = all_data.min()
    max_d = all_data.max()
    return min_d, max_d

#正規化_minmax
def normalize_minmax(min_, max_, d):
    min_max_d = (d - min_) / (max_ - min_)
    return min_max_d    

In [50]:
min_, max_ = get_minmax(200,300)

In [55]:
%%time
for i in range(736,737):
    d = np.loadtxt(rf"Z:\200 実験データ加工\20220630\TrainingData\FFT\csv\FFT_{i}.csv", delimiter=',')  # 区切り文字としてカンマを指定
    d_norm = normalize_minmax(min_,max_,d)
    amp_db_norm = pd.DataFrame({'db': d_norm})
    output_csv = rf"Z:\200 実験データ加工\20220630\TrainingData\FFT_Norm\csv\FFT_Norm_{i}.csv"
    amp_db_norm.to_csv(output_csv, header=False, index=False)

Wall time: 62 ms


# -------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------
# 2021年11月

# 検証データ(test data)作成

## ファイル名変更 ex) AE_18.tdms -> AE_1.tdms
### プログラム書きやすいように

In [59]:
for i in range(24,506,2):
    num = int((i-22)/2)
    old_name = rf"Z:\100 実験データ（RAW）\20211115---切削実験---AE+切削抵抗\AE\1-5000\AE_{i}.tdms"
    new_name = rf"Z:\100 実験データ（RAW）\20211115---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    os.rename(old_name, new_name)
#     print(i,num)

# ①生データ から2^14個取ってくる
## 中間データ①

In [63]:
# AE_{num}.tdms -> AE_extract_{num}.csv
# 生データ から2^14個取ってくる

def extract_16384(num):
    load_path = rf"Z:\100 実験データ（RAW）\20211115---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    save_path = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\extract\AE_extract_{num}.csv"
    d = load_tdms(load_path, ch_num=3)
    d = d[0][5000000:5000000+16384]
    pd.DataFrame(d).to_csv(save_path)

In [65]:
%%time
for i in range(1,242):
    extract_16384(i)

Wall time: 9min 41s


# ②正規化する前のFFT
## 中間データ②作成

In [66]:
def FFT_withoutNorm(num):
    
    #データ読み込み
    d = pd.read_csv(rf'Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\extract\AE_extract_{num}.csv')
    d = d['0']
    
    #FFT条件
    dt = 1/10**6
    split_t_r = 0.1 # 1つの枠で全体のどの割合のデータを分析するか。
    overlap = 0.5  # オーバーラップ率
    window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
    y_label = "amplitude"
    y_unit = "V"
    N = len(d)
    fq = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
    

    #FFT実行
    fq_, amp_ = FFT_main(fq, d, dt, split_t_r, overlap, window_F)

    #fig保存ー正規化前
    output_FN = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\fig\FFT_{num}"
    plot_FFT(fq_, amp_, output_FN, y_label, y_unit,num)

    #dBに変換してcsv保存ー正規化前
    amp_db = pd.DataFrame({'db': 20*np.log10(amp_)})
    output_csv = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\csv\FFT_{num}.csv"
    amp_db.to_csv(output_csv, header=False, index=False)

In [67]:
%%time
for i in range(1,242):
    FFT_withoutNorm(i)

Wall time: 2min 6s


# FFT(正規化）
## 検証（test）データ作成

In [68]:
#学習データから最大最小を算出
#今回は2022年6月の200path-300path
def get_minmax(start,end):
    all_data = []
    for i in range(start,end+1):
        d = pd.read_csv(rf"Z:\200 実験データ加工\20220630\TrainingData\FFT\csv\FFT_{i}.csv")
        d = d.to_numpy()
        d = np.ravel(d)
        all_data.append(d[:])
    all_data = np.array(all_data)
    all_data = all_data.ravel()
    min_d = all_data.min()
    max_d = all_data.max()
    return min_d, max_d

#正規化_minmax
def normalize_minmax(min_, max_, d):
    min_max_d = (d - min_) / (max_ - min_)
    return min_max_d    

In [69]:
min_, max_ = get_minmax(200,300)

In [70]:
%%time
for i in range(1,242):
    d = np.loadtxt(rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\csv\FFT_{i}.csv", delimiter=',')  # 区切り文字としてカンマを指定
    d_norm = normalize_minmax(min_,max_,d)
    amp_db_norm = pd.DataFrame({'db': d_norm})
    output_csv = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT_Norm\csv\FFT_Norm_{i}.csv"
    amp_db_norm.to_csv(output_csv, header=False, index=False)

Wall time: 6.42 s
